# 소녀시대 멤버 얼굴 분류기

1. 데이터셋 수집: 윤아와 써니 사진 1000여장을 크롤링을 통해 구하였다. 
2. 이미지 파일 수치화
3. 이미지데이터셋 클래스 생성
4. 모델구축
5. 학습 및 검증


## 데이터셋 수집 및 전처리

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os
from urllib.request import urlretrieve
from tqdm import tqdm

In [ ]:
# 이미지 크롤링

keyword = input("keyword: ")
url = "https://search.naver.com/search.naver?where=image&sm=tab_jum&query={}".format(keyword)

print("접속중")
driver = webdriver.Chrome("/Users/jungyeonkoh/Downloads/chromedriver")
driver.implicitly_wait(30)
driver.get(url)

# 페이지 스크롤 다운
body = driver.find_element_by_css_selector("body")
for i in range(150):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

imgs = driver.find_elements_by_css_selector("img._img")
result = []
for img in tqdm(imgs):
    if "http" in img.get_attribute("src"):
        result.append(img.get_attribute("src"))

driver.close()
print("수집완료")

# 폴더 생성
print("폴더생성")
if not os.path.isdir("/Users/jungyeonkoh/Desktop/YAI/Crawling_Images/{}".format(keyword)):
    os.mkdir("/Users/jungyeonkoh/Desktop/Crawling_Images/{}".format(keyword))

# 다운로드
for index, link in tqdm(enumerate(result)):
    start = link.rfind('.')
    end = link.rfind('&')
    filetype = link[start:end]
    
    urlretrieve(link, "/Users/jungyeonkoh/Desktop/YAI/Crawling_Images/{}/{}{}{}".format(keyword, keyword, index, filetype))
    time.sleep(1)

print("다운로드완료")

In [ ]:
import os
import shutil

# 불필요한 이미지 삭제 후 라벨링
file_path = "/Users/jungyeonkoh/Desktop/YAI/Crawling_Images/티파니"
test_dir = "/Users/jungyeonkoh/Desktop/YAI/Crawling_Images/윤아_testdata"
file_names = os.listdir(file_path)

i = 1
for name in file_names:
    start = name.rfind('.')
    filetype = name[start:]
    src = os.path.join(file_path, name)
    dst = str(i) + ".jpg"
    dst = os.path.join(file_path, dst)
    os.rename(src, dst)
    i += 1

### Crop_Face

In [ ]:
from googleapiclient import  discovery
from oauth2client.client  import GoogleCredentials
import sys
import io
import base64
from PIL import Image
from PIL import ImageDraw
from genericpath import isfile
import os
import hashlib
from oauth2client.service_account import ServiceAccountCredentials
import imageio
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]='/Users/jungyeonkoh/Downloads/My First Project-500b6dfc89fb.json'

def detect_faces(path):
    """Detects faces in an image."""
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.types.Image(content=content)

    response = client.face_detection(image=image)
    faces = response.face_annotations
    
    
    # Names of likelihood from google.cloud.vision.enums
    likelihood_name = ('UNKNOWN', 'VERY_UNLIKELY', 'UNLIKELY', 'POSSIBLE',
                       'LIKELY', 'VERY_LIKELY')
    # print('Faces:')
    """
    for face in faces:
        # print('anger: {}'.format(likelihood_name[face.anger_likelihood]))
        # print('joy: {}'.format(likelihood_name[face.joy_likelihood]))
        # print('surprise: {}'.format(likelihood_name[face.surprise_likelihood]))

        vertices = ([(vertex.x, vertex.y)
                    for vertex in face.bounding_poly.vertices])

        # print('face bounds: {}'.format(','.join(vertices)))
    """
    vertices = ([(vertex.x, vertex.y) for vertex in faces[0].bounding_poly.vertices])
    
    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))

    return vertices

In [ ]:
def crop_face(path, label):
    for i, image in enumerate(os.listdir(path+label)):
        try:
            face = detect_faces(path+label+'/'+image)
            img = imageio.imread(path+label+'/'+image)
            cropped = img[face[0][1]:face[2][1], face[0][0]:face[1][0],:]

            ### make folder and save picture in that directory
            saveDir = path+label+"_cropped"
            try:
                if not(os.path.isdir(saveDir)):
                    os.makedirs(os.path.join(saveDir))
            except OSError as e:
                if e.errno != errno.EEXIST:
                    print("Failed to create directory!!!!!")
                    raise

            ### save
            im = Image.fromarray(cropped)
            im.save(saveDir+'/'+str(i+306)+'.jpg')
            if i%100 == 0:
                print(str(i)+'/'+str(len(os.listdir(path+label)))+' images cropped')
        except:
            print(str(i)+'th image '+path+label+'/'+image+' passed~')
            pass

    print('Total '+str(len(os.listdir(path+label)))+' images cropped')

In [ ]:
# crop할 이미지들이 들어있는 폴더
path = "/Users/jungyeonkoh/Desktop/YAI/Crawling_Images/"
label= "써니"

crop_face(path, label)

In [ ]:
import matplotlib.pyplot as plt

# 언제쓸지모르지만 이미지 array 시각화
def show_img(idx):
    plt.rcParams["figure.figsize"] = (10, 10) # set figure size
    plt.imshow(train[idx])
    plt.show()

### Transformation & DataLoader

In [ ]:
import torch, torchvision
from torchvision import models, transforms, datasets
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
from PIL import Image
import os

In [ ]:
import splitfolders
splitfolders.fixed("./Crawling_Images", "./", fixed = (0,50))

In [ ]:
trans = transforms.Compose([transforms.Resize((96,96)), transforms.ToTensor(), 
                            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [ ]:
batch_size = 50
train_directory = "/Users/jungyeonkoh/Desktop/YAI/train"
test_directory = "/Users/jungyeonkoh/Desktop/YAI/test"

data = {"train": datasets.ImageFolder(root = train_directory, transform = trans),
        "test": datasets.ImageFolder(root = test_directory, transform = trans)}

idx_to_class = {v : k for k, v in data["train"].class_to_idx.items()}

train_data_loader = DataLoader(data["train"], batch_size = batch_size, shuffle = True)
test_data_loader = DataLoader(data["test"], batch_size = batch_size, shuffle = True)

## CNN 모델 구축 및 학습

In [ ]:
import os
import google.colab
from google.colab import drive

drive.mount("/content/drive")
os.chdir("/content/drive/My Drive")

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

In [ ]:
# CNN 구현: Resnet50
model = models.resnet50(num_classes = 8)
model = model.to(device)

print(model)

# init
learning_rate = 0.002
batch_size = 15
epoch = 20
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
# 모델 학습
train_data_size = len(data["train"])
history = []

for i in range(epoch):
    model.train()
    train_loss = 0.0
    train_acc = 0.0

    for num, (inputs, labels) in enumerate(train_data_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

        #  label값과 prediction값 비교
        val, predictions = torch.max(outputs.data, 1)
        correct_counts = predictions.eq(labels.data.view_as(predictions))

        # train_loss, train_accuracy
        acc = torch.mean(correct_counts.type(torch.FloatTensor))
        train_acc += acc.item() * inputs.size(0)

        print("Batch Number: {}, Training Loss: {:.4f}, Accuracy: {:.4f}".format(num, loss.item(), acc.item()))

    avg_train_loss = train_loss / train_data_size
    avg_train_acc = train_acc / train_data_size
    history.append([avg_train_loss, avg_train_acc])
    print("Epoch: {}, Training Loss: {:.4f}, Accuracy: {:.4f}".format(i, avg_train_loss, avg_train_acc))

print("학습 완료")

In [ ]:
# visualization

import matplotlib.pyplot as plt

h = np.array(history)

plt.plot(h[:,0])
plt.xlabel("Epoch Number")
plt.ylabel("avg_train_loss")
plt.show()

plt.plot(h[:,1])
plt.xlabel("Epoch Number")
plt.ylabel("avg_train_accuracy")
plt.show()

In [ ]:
# test_data를 통한 검증

test_acc = 0.0
test_loss = 0.0
test_data_size = len(data["test"])

with torch.no_grad():
  model.eval()

  for num, (inputs, labels) in enumerate(test_data_loader):
    inputs = inputs.to(device)
    labels = labels.to(device)

    outputs = model(inputs)
    loss = loss_func(outputs, labels)
    test_loss += loss.item() * inputs.size(0)
    val, predictions = torch.max(outputs.data, 1)
    correct_counts = predictions.eq(labels.data.view_as(predictions))

    acc = torch.mean(correct_counts.type(torch.FloatTensor))
    test_acc += acc.item() * inputs.size(0)

    print("Test Batch Number: {}, Test Loss: {:.4f}, Accuracy: {:.4f}".format(num, loss.item(), acc.item()))
  
  avg_test_loss = test_loss / test_data_size
  avg_test_acc = test_acc / test_data_size

  print("Test Accuracy: " + str(avg_test_acc))

In [ ]:
# 개별 파일에 대한 예측

test_image_path = "./test/윤아/140.jpg"
test_image = Image.open(test_image_path)
plt.imshow(test_image)

test_image_tensor = trans(test_image)
test_image_tensor = test_image_tensor.view(1,3,96,96).cuda()

with torch.no_grad():
  model.eval()
  out = model(test_image_tensor)
  ps = torch.exp(out)
  topk, topclass = ps.topk(3, dim = 1)
  for i in range(3):
    print("Prediction ", i + 1, ": ", idx_to_class[topclass.cpu().numpy()[0][i]], ", Score: ", topk.cpu().numpy()[0][i])